In [2]:
import googlemaps
import html
import re
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
skt_api_key = os.getenv("SKT_API_KEY")

gmaps = googlemaps.Client(key=google_api_key)

## 도착역과 출구 가져오기

In [4]:
def get_station_exit(origin, destination):
    time = datetime(2023, 6, 30, 13, 30, 0)
    # 경로 검색
    directions_result = gmaps.directions(origin,
                                         destination,
                                         mode="transit",
                                         transit_mode='subway',
                                         departure_time=time,
                                         language='ko')
    # 경로의 단계 출력
    for route in directions_result:
        for leg in route['legs']:
            for step in leg['steps']:
                if 'html_instructions' in step:
                    html_instructions = html.unescape(step['html_instructions'])
#                     print(html_instructions)
                if 'transit_details' in step:
                    transit_details = step['transit_details']
                    line_name = transit_details['line']['short_name']
                    departure_stop = transit_details['departure_stop']['name']
                    arrival_stop = transit_details['arrival_stop']['name']
                    final_station = arrival_stop + '역'
#                     print(f"지하철 {line_name}을(를) {departure_stop}에서 탑승하여 {arrival_stop}에서 하차합니다.")
                if 'steps' in step:
                    for inner_step in step['steps']:
                        if 'html_instructions' in inner_step:
                            html_instructions = html.unescape(inner_step['html_instructions'])
                            # 정규식을 이용하여 숫자 추출
                            exit_num = re.findall(r"\d+", html_instructions)
#                             print(html_instructions)
    result = {
        'final_station' : final_station,
        'exit_num' : exit_num[0]
    }
    
    return result

In [5]:
import pandas as pd

loc_data = pd.read_csv('source/cherryblossom_loc_selected.csv',
                     sep=',', engine='python')

final_station_list = []
exit_num_list = []

for index, row in loc_data.iterrows():
    result = get_station_exit('성신여대입구역(돈암) 2번출구', row['address'])
    if result['final_station'] == '보라매공원역':
        result['final_station'] = '보라매역'
    elif result['final_station'] == '무악재역역':
        result['final_station'] = '무악재역'
    elif result['final_station'] == '중랑역':
        result['final_station'] = '외대앞역'
    elif result['final_station'] == '면목역':
        result['final_station'] = '외대앞역'

    final_station_list.append(result['final_station'])
    exit_num_list.append(result['exit_num'])

loc_data['final_station']=final_station_list
loc_data['exit_num']=exit_num_list

loc_data

loc_name                      address    gu  main_flower  \
0             여의 동·서로(윤중로)         서울특별시 영등포구 여의도동 여의서로  영등포구           벚꽃   
1   올림픽로(광진교 남단 ~ LG전자사거리)           서울특별시 강동구 올림픽로 722   강동구           벚꽃   
2        벚꽃로(독산역~가산디지털단지역)             서울특별시 금천구 벚꽃로 30   금천구           벚꽃   
3             송파나루공원(석촌호수)          서울특별시 송파구 석촌호수로 188   송파구  벚꽃, 철쭉, 야생화   
4                    보라매공원  서울특별시 동작구 여의대방로20길 33 보라매공원   동작구           벚꽃   
5     안산도시자연공원 (안산문화쉼터벚꽃길)        서울특별시 서대문구 연희로32길 134  서대문구  벚꽃, 철쭉, 개나리   
6                    경희대학교           서울특별시 동대문구 경희대로 24  동대문구    벚꽃, 개나리 등   
7                     방화공원           서울특별시 강서구 금낭화로 167   강서구           벚꽃   
8                 서울숲 벚나무길      서울특별시 성동구 뚝섬로 273 서울숲공원   성동구       벚꽃, 튤립   
9                      오리로             서울특별시 구로구 경인로 98   구로구           벚꽃   
10             도구로(도구머리꽃길)           서울특별시 서초구 도구로 45-3   서초구      벚꽃, 개나리   
11              어린이대공원 산책로            서울특별시 광진구 능동로 216   광진구           벚꽃   
12     희우정로(양화로11 ~망원정사거리)            서울특별시 마포구 희우정로 66   마포구           벚꽃   
13                    워커힐길           서울특별시 광진구 광장동 워커힐길   광진구           벚꽃   
14          도림천(신대방역~시흥대로)              서울특별시 동작구 대림로 2   관악구           벚꽃   
15          경의선숲길(공덕역~대흥역)         서울특별시 마포구 백범로36길 8-7   마포구           벚꽃   
16                남산공원 순환로           서울특별시 중구 남산공원길 507    중구         벚꽃 등   
17       성내천 둔치(위례성길~복개종점)             서울특별시 송파구 성내천로 6   송파구           벚꽃   
18                    양재천변            서울특별시 서초구 양재동 양재천   서초구    개나리, 벚꽃 등   
19                  경춘선 숲길          서울특별시 노원구 화랑로51가길 3   노원구           벚꽃   
20         북서울 꿈의 숲 월영지 주변   서울특별시 강북구 월계로 173 북서울 꿈의 숲   강북구           벚꽃   
21         안양천 제방(신정교~양평교)   서울특별시 영등포구 양평동 487 안양천 제방길  영등포구  벚꽃, 개나리, 철쭉   
22          중랑천제방(공원녹지순환길)   서울특별시 중랑구 중화동 410-11 중랑천제방  동대문구    벚꽃, 개나리 등   

      season length                                            feature  \
0    3월말~4월중   6.9㎞  여의도 국회주변 화려한 벚꽃이 장관을 이루며 매년 ‘한강 여의도 봄꽃축제’ 개최(나...   
1    4월중~4월말  0.7km                                   가로변 벚꽃이 아름다움(산책)   
2    4월중~4월말   3.4㎞                     십리에 이르는 철도가로변에 벚꽃길이 아름다움(드라이브)   
3    4월중~6월초  2.5km            석촌호수와 어우러져 피는 벚꽃, 철쭉, 야생화가 아름다움(산책, 운동)   
4         4월  0.2km  공원 내 전시된 공군항공기와 봄철 화려하게 핀 벚꽃을  한눈에 담을 수 있는 경관이...   
5    4월초~4월말   2.0㎞     안산 입구로 왕벚나무가 만개할 때 드라마 촬영지로 각광을 받고 있음(나들이, 산책)   
6    4월초~6월중  2.0km                      대학교 내 3대 명건물과 함께  아름다운 벚꽃길 연출   
7    4월중~4월말   2.0㎞                            개화산과 연계되어 봄꽃이 아름다움(나들이)   
8    4월초～5월중   0.7㎞                   연분홍 벚꽃과 화사한 색감의 튤립이 돋보임(나들이, 산책)   
9        4월중  1.0km                   궁동생태공원 인근 벚나무가 식재된 거리 꽃길을 조망(산책)   
10   4월중~4월말  0.4km                 도구머리공원 사잇길을 따라 한적하게 산책하기 좋은 꽃길(산책)   
11   4월중~4월말   1.5㎞             벚꽃길이 아름다워 가족, 연인들의 나들이 장소로 최적(나들이, 산책)   
12   4월초~4월중  0.5km                      주민들의 산책로에 벚꽃으로 아름다운 경관 제공(산책)   
13   4월초~4월중   1.5㎞             연분홍색과 벚꽃과 아차산의 경관이 어우러져 아름다움 (드라이브,산책)   
14   4월중~4월말  1.0km               제방위 산책로에 벚꽃이 만개하면 아름다운 경관 연출(산책, 운동)   
15   3월말~4월말   0.8㎞      산책로를 따라 피어난 벚꽃이 다양한 수목들과 어우러져 아름다움을 경관 연출(산책)   
16   4월중~4월말   5.0㎞                      연분홍 벚꽃길이 아름다운 경관을 제공(운동, 나들이)   
17   4월중~4월말  1.8km  주민헌수를 통하여 식재한 왕벚나무 400여주가 만개하여 아름다운 경관을 연출함(산책...   
18   4월초～4월말   2.5㎞                   하천변 산책로를 따라 봄꽃을 감상할 수 있음(산책, 운동)   
19  3월말~4월중순  0.4km                             철길을 따라 아름다운 벚꽃길 연출(산책)   
20   4월중~4월말  4.0km      연분홍색의 벚꽃이 월영지, 월광폭포와 어울려 아름다운 경관을 제공(나들이, 산책)   
21   3월말~4월중   3.0㎞   안양천을 따라 산책로가 조성되어 있고 산책로변으로 꽃이 아름다움(나들이, 산책, 운동)   
22   4월초~4월중   3.0㎞                   공원녹지 순환길을 따라 아름다운 벚꽃길 연출(산책, 운동)   

    result_stats final_station exit_num  
0        1360000        국회의사당역        2  
1        1230000           천호역        3  
2         980000         금천구청역        1  
3         687000           잠실역        3  
4         401000          보라매역        1  
5         363000          무악재역        4  
6         246000           회기역        1  
7         245000           방화역  

## 역사코드 추가

In [6]:
import requests

url = "https://apis.openapi.sk.com/puzzle/subway/meta/stations?type=exit"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": skt_api_key
}

response = requests.get(url, headers=headers)

In [7]:
data = response.json()
station_code_list = []

for index, row in loc_data.iterrows():
    for content in data['contents']:
        if content['stationName'] == row['final_station']:
            station_code_list.append(content['stationCode'])

loc_data['station_code'] = station_code_list
loc_data

loc_name                      address    gu  main_flower  \
0             여의 동·서로(윤중로)         서울특별시 영등포구 여의도동 여의서로  영등포구           벚꽃   
1   올림픽로(광진교 남단 ~ LG전자사거리)           서울특별시 강동구 올림픽로 722   강동구           벚꽃   
2        벚꽃로(독산역~가산디지털단지역)             서울특별시 금천구 벚꽃로 30   금천구           벚꽃   
3             송파나루공원(석촌호수)          서울특별시 송파구 석촌호수로 188   송파구  벚꽃, 철쭉, 야생화   
4                    보라매공원  서울특별시 동작구 여의대방로20길 33 보라매공원   동작구           벚꽃   
5     안산도시자연공원 (안산문화쉼터벚꽃길)        서울특별시 서대문구 연희로32길 134  서대문구  벚꽃, 철쭉, 개나리   
6                    경희대학교           서울특별시 동대문구 경희대로 24  동대문구    벚꽃, 개나리 등   
7                     방화공원           서울특별시 강서구 금낭화로 167   강서구           벚꽃   
8                 서울숲 벚나무길      서울특별시 성동구 뚝섬로 273 서울숲공원   성동구       벚꽃, 튤립   
9                      오리로             서울특별시 구로구 경인로 98   구로구           벚꽃   
10             도구로(도구머리꽃길)           서울특별시 서초구 도구로 45-3   서초구      벚꽃, 개나리   
11              어린이대공원 산책로            서울특별시 광진구 능동로 216   광진구           벚꽃   
12     희우정로(양화로11 ~망원정사거리)            서울특별시 마포구 희우정로 66   마포구           벚꽃   
13                    워커힐길           서울특별시 광진구 광장동 워커힐길   광진구           벚꽃   
14          도림천(신대방역~시흥대로)              서울특별시 동작구 대림로 2   관악구           벚꽃   
15          경의선숲길(공덕역~대흥역)         서울특별시 마포구 백범로36길 8-7   마포구           벚꽃   
16                남산공원 순환로           서울특별시 중구 남산공원길 507    중구         벚꽃 등   
17       성내천 둔치(위례성길~복개종점)             서울특별시 송파구 성내천로 6   송파구           벚꽃   
18                    양재천변            서울특별시 서초구 양재동 양재천   서초구    개나리, 벚꽃 등   
19                  경춘선 숲길          서울특별시 노원구 화랑로51가길 3   노원구           벚꽃   
20         북서울 꿈의 숲 월영지 주변   서울특별시 강북구 월계로 173 북서울 꿈의 숲   강북구           벚꽃   
21         안양천 제방(신정교~양평교)   서울특별시 영등포구 양평동 487 안양천 제방길  영등포구  벚꽃, 개나리, 철쭉   
22          중랑천제방(공원녹지순환길)   서울특별시 중랑구 중화동 410-11 중랑천제방  동대문구    벚꽃, 개나리 등   

      season length                                            feature  \
0    3월말~4월중   6.9㎞  여의도 국회주변 화려한 벚꽃이 장관을 이루며 매년 ‘한강 여의도 봄꽃축제’ 개최(나...   
1    4월중~4월말  0.7km                                   가로변 벚꽃이 아름다움(산책)   
2    4월중~4월말   3.4㎞                     십리에 이르는 철도가로변에 벚꽃길이 아름다움(드라이브)   
3    4월중~6월초  2.5km            석촌호수와 어우러져 피는 벚꽃, 철쭉, 야생화가 아름다움(산책, 운동)   
4         4월  0.2km  공원 내 전시된 공군항공기와 봄철 화려하게 핀 벚꽃을  한눈에 담을 수 있는 경관이...   
5    4월초~4월말   2.0㎞     안산 입구로 왕벚나무가 만개할 때 드라마 촬영지로 각광을 받고 있음(나들이, 산책)   
6    4월초~6월중  2.0km                      대학교 내 3대 명건물과 함께  아름다운 벚꽃길 연출   
7    4월중~4월말   2.0㎞                            개화산과 연계되어 봄꽃이 아름다움(나들이)   
8    4월초～5월중   0.7㎞                   연분홍 벚꽃과 화사한 색감의 튤립이 돋보임(나들이, 산책)   
9        4월중  1.0km                   궁동생태공원 인근 벚나무가 식재된 거리 꽃길을 조망(산책)   
10   4월중~4월말  0.4km                 도구머리공원 사잇길을 따라 한적하게 산책하기 좋은 꽃길(산책)   
11   4월중~4월말   1.5㎞             벚꽃길이 아름다워 가족, 연인들의 나들이 장소로 최적(나들이, 산책)   
12   4월초~4월중  0.5km                      주민들의 산책로에 벚꽃으로 아름다운 경관 제공(산책)   
13   4월초~4월중   1.5㎞             연분홍색과 벚꽃과 아차산의 경관이 어우러져 아름다움 (드라이브,산책)   
14   4월중~4월말  1.0km               제방위 산책로에 벚꽃이 만개하면 아름다운 경관 연출(산책, 운동)   
15   3월말~4월말   0.8㎞      산책로를 따라 피어난 벚꽃이 다양한 수목들과 어우러져 아름다움을 경관 연출(산책)   
16   4월중~4월말   5.0㎞                      연분홍 벚꽃길이 아름다운 경관을 제공(운동, 나들이)   
17   4월중~4월말  1.8km  주민헌수를 통하여 식재한 왕벚나무 400여주가 만개하여 아름다운 경관을 연출함(산책...   
18   4월초～4월말   2.5㎞                   하천변 산책로를 따라 봄꽃을 감상할 수 있음(산책, 운동)   
19  3월말~4월중순  0.4km                             철길을 따라 아름다운 벚꽃길 연출(산책)   
20   4월중~4월말  4.0km      연분홍색의 벚꽃이 월영지, 월광폭포와 어울려 아름다운 경관을 제공(나들이, 산책)   
21   3월말~4월중   3.0㎞   안양천을 따라 산책로가 조성되어 있고 산책로변으로 꽃이 아름다움(나들이, 산책, 운동)   
22   4월초~4월중   3.0㎞                   공원녹지 순환길을 따라 아름다운 벚꽃길 연출(산책, 운동)   

    result_stats final_station exit_num station_code  
0        1360000        국회의사당역        2          914  
1        1230000           천호역        3          547  
2         980000         금천구청역        1         P144  
3         687000           잠실역        3          216  
4         401000          보라매역        1          742  
5         363000          무악재역        

## 위도, 경도 정보 추가

In [10]:
lat_list=[]
long_list=[]

for index, row in loc_data.iterrows():
    geocode_result = gmaps.geocode(row['address'])
    if len(geocode_result) > 0:
        lat = geocode_result[0]['geometry']['location']['lat']
        long = geocode_result[0]['geometry']['location']['lng']
        lat_list.append(lat)
        long_list.append(long)

loc_data['lat'] = lat_list
loc_data['long'] = long_list

loc_data

loc_name                      address    gu  main_flower  \
0             여의 동·서로(윤중로)         서울특별시 영등포구 여의도동 여의서로  영등포구           벚꽃   
1   올림픽로(광진교 남단 ~ LG전자사거리)           서울특별시 강동구 올림픽로 722   강동구           벚꽃   
2        벚꽃로(독산역~가산디지털단지역)             서울특별시 금천구 벚꽃로 30   금천구           벚꽃   
3             송파나루공원(석촌호수)          서울특별시 송파구 석촌호수로 188   송파구  벚꽃, 철쭉, 야생화   
4                    보라매공원  서울특별시 동작구 여의대방로20길 33 보라매공원   동작구           벚꽃   
5     안산도시자연공원 (안산문화쉼터벚꽃길)        서울특별시 서대문구 연희로32길 134  서대문구  벚꽃, 철쭉, 개나리   
6                    경희대학교           서울특별시 동대문구 경희대로 24  동대문구    벚꽃, 개나리 등   
7                     방화공원           서울특별시 강서구 금낭화로 167   강서구           벚꽃   
8                 서울숲 벚나무길      서울특별시 성동구 뚝섬로 273 서울숲공원   성동구       벚꽃, 튤립   
9                      오리로             서울특별시 구로구 경인로 98   구로구           벚꽃   
10             도구로(도구머리꽃길)           서울특별시 서초구 도구로 45-3   서초구      벚꽃, 개나리   
11              어린이대공원 산책로            서울특별시 광진구 능동로 216   광진구           벚꽃   
12     희우정로(양화로11 ~망원정사거리)            서울특별시 마포구 희우정로 66   마포구           벚꽃   
13                    워커힐길           서울특별시 광진구 광장동 워커힐길   광진구           벚꽃   
14          도림천(신대방역~시흥대로)              서울특별시 동작구 대림로 2   관악구           벚꽃   
15          경의선숲길(공덕역~대흥역)         서울특별시 마포구 백범로36길 8-7   마포구           벚꽃   
16                남산공원 순환로           서울특별시 중구 남산공원길 507    중구         벚꽃 등   
17       성내천 둔치(위례성길~복개종점)             서울특별시 송파구 성내천로 6   송파구           벚꽃   
18                    양재천변            서울특별시 서초구 양재동 양재천   서초구    개나리, 벚꽃 등   
19                  경춘선 숲길          서울특별시 노원구 화랑로51가길 3   노원구           벚꽃   
20         북서울 꿈의 숲 월영지 주변   서울특별시 강북구 월계로 173 북서울 꿈의 숲   강북구           벚꽃   
21         안양천 제방(신정교~양평교)   서울특별시 영등포구 양평동 487 안양천 제방길  영등포구  벚꽃, 개나리, 철쭉   
22          중랑천제방(공원녹지순환길)   서울특별시 중랑구 중화동 410-11 중랑천제방  동대문구    벚꽃, 개나리 등   

      season length                                            feature  \
0    3월말~4월중   6.9㎞  여의도 국회주변 화려한 벚꽃이 장관을 이루며 매년 ‘한강 여의도 봄꽃축제’ 개최(나...   
1    4월중~4월말  0.7km                                   가로변 벚꽃이 아름다움(산책)   
2    4월중~4월말   3.4㎞                     십리에 이르는 철도가로변에 벚꽃길이 아름다움(드라이브)   
3    4월중~6월초  2.5km            석촌호수와 어우러져 피는 벚꽃, 철쭉, 야생화가 아름다움(산책, 운동)   
4         4월  0.2km  공원 내 전시된 공군항공기와 봄철 화려하게 핀 벚꽃을  한눈에 담을 수 있는 경관이...   
5    4월초~4월말   2.0㎞     안산 입구로 왕벚나무가 만개할 때 드라마 촬영지로 각광을 받고 있음(나들이, 산책)   
6    4월초~6월중  2.0km                      대학교 내 3대 명건물과 함께  아름다운 벚꽃길 연출   
7    4월중~4월말   2.0㎞                            개화산과 연계되어 봄꽃이 아름다움(나들이)   
8    4월초～5월중   0.7㎞                   연분홍 벚꽃과 화사한 색감의 튤립이 돋보임(나들이, 산책)   
9        4월중  1.0km                   궁동생태공원 인근 벚나무가 식재된 거리 꽃길을 조망(산책)   
10   4월중~4월말  0.4km                 도구머리공원 사잇길을 따라 한적하게 산책하기 좋은 꽃길(산책)   
11   4월중~4월말   1.5㎞             벚꽃길이 아름다워 가족, 연인들의 나들이 장소로 최적(나들이, 산책)   
12   4월초~4월중  0.5km                      주민들의 산책로에 벚꽃으로 아름다운 경관 제공(산책)   
13   4월초~4월중   1.5㎞             연분홍색과 벚꽃과 아차산의 경관이 어우러져 아름다움 (드라이브,산책)   
14   4월중~4월말  1.0km               제방위 산책로에 벚꽃이 만개하면 아름다운 경관 연출(산책, 운동)   
15   3월말~4월말   0.8㎞      산책로를 따라 피어난 벚꽃이 다양한 수목들과 어우러져 아름다움을 경관 연출(산책)   
16   4월중~4월말   5.0㎞                      연분홍 벚꽃길이 아름다운 경관을 제공(운동, 나들이)   
17   4월중~4월말  1.8km  주민헌수를 통하여 식재한 왕벚나무 400여주가 만개하여 아름다운 경관을 연출함(산책...   
18   4월초～4월말   2.5㎞                   하천변 산책로를 따라 봄꽃을 감상할 수 있음(산책, 운동)   
19  3월말~4월중순  0.4km                             철길을 따라 아름다운 벚꽃길 연출(산책)   
20   4월중~4월말  4.0km      연분홍색의 벚꽃이 월영지, 월광폭포와 어울려 아름다운 경관을 제공(나들이, 산책)   
21   3월말~4월중   3.0㎞   안양천을 따라 산책로가 조성되어 있고 산책로변으로 꽃이 아름다움(나들이, 산책, 운동)   
22   4월초~4월중   3.0㎞                   공원녹지 순환길을 따라 아름다운 벚꽃길 연출(산책, 운동)   

    result_stats final_station exit_num station_code        lat        long  
0        1360000        국회의사당역        2          914  37.533101  126.919997  
1        1230000           천호역        3          547  37.545675  127.126219  
2         980000         금천구청역        1         P144  37.458239  126.894177  
3         687000           잠실역        3          216  37

In [11]:
loc_data.to_csv('source/cherryblossom_loc_selected_station.csv',
                      encoding='cp949', mode='w', index=False)